In [7]:
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
from kdutils.common import fetch_temp_data, fetch_temp_returns

In [9]:
method = 'cicso0'
instruments = 'ims'
task_id = '200037'
period = 15
name = 'lgbm'

#### 提取基础数据

In [10]:
total_data = fetch_temp_data(
        method=method,
        instruments=instruments,
        task_id=task_id,
        datasets=['train'])
total_data.head()

./records/cicso0/ims/level2/train_data.feather


,trade_time,symbol,open,high,low,close,volume,money,twap,pct_change,...,price_imbalance_3,price_imbalance_4,depth_imbalance_0,depth_imbalance_1,depth_imbalance_2,depth_imbalance_3,depth_imbalance_4,ask_bid_press,realized_volatility,code
0,2022-07-25 09:31:00,IM2208,6971.8,6994.8,6964.4,6973.8,602,840545880.0,6981.585000,0.000287,...,-0.000548,-0.000727,0.048350,0.032361,0.080896,0.116902,-0.009520,0.191198,0.002546,IM
1,2022-07-25 09:32:00,IM2208,6973.8,6995.2,6969.6,6982.8,428,597671840.0,6982.126050,0.001291,...,-0.000485,-0.000618,0.077734,0.073275,0.029830,0.017027,0.020993,0.243969,0.001839,IM
2,2022-07-25 09:33:00,IM2208,6982.8,6989.8,6981.6,6989.8,280,391195160.0,6985.326496,0.001002,...,-0.000393,-0.000494,0.072032,0.123311,0.126821,0.172182,0.129718,0.337424,0.001727,IM
3,2022-07-25 09:34:00,IM2208,6989.6,6993.0,6982.6,6993.0,319,445873560.0,6988.716949,0.000486,...,-0.000357,-0.000442,-0.014664,-0.053867,0.042050,-0.065601,-0.069451,-0.099565,0.001665,IM
4,2022-07-25 09:35:00,IM2208,6993.6,7009.0,6991.8,7002.2,284,397659840.0,6999.337288,0.001230,...,-0.000445,-0.000558,0.056584,0.106953,0.097129,0.101817,0.024232,0.132744,0.001700,IM


#### 1. 聚合方式bar 计算收益率

In [11]:
### 聚合bar
basic_columns = [
        'close', 'high', 'low', 'open', 'volume'
    ]
aggregation_rules = {
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum',
        'volume': 'sum',  # 成交量是流量，用 'sum'
    }
market_data = total_data[['trade_time', 'code'] + basic_columns]
market_data_indexed = market_data.set_index('trade_time')
agg_market_data = market_data_indexed.resample(
        '{0}T'.format(period), label='right',
        closed='right').agg(aggregation_rules)
agg_market_data.head()

,open,high,low,close,volume
trade_time,,,,,
2022-07-25 09:45:00,6971.8,7015.8,6964.4,7001.8,3666
2022-07-25 10:00:00,7001.8,7004.6,6944.0,6955.0,2385
2022-07-25 10:15:00,6955.2,6978.2,6944.4,6953.6,1154
2022-07-25 10:30:00,6953.6,6973.6,6938.0,6971.8,1055
2022-07-25 10:45:00,6971.8,6974.2,6956.8,6961.8,788


In [24]:
return1_data = ((agg_market_data['close'].shift(-16) / agg_market_data['close'].shift(-1)) - 1).head(5)
return1_data.head()

trade_time
2022-07-25 09:45:00   -0.009863
2022-07-25 10:00:00   -0.011419
2022-07-25 10:15:00   -0.012049
2022-07-25 10:30:00   -0.011290
2022-07-25 10:45:00   -0.011142
Freq: 15T, Name: close, dtype: float64

#### 2.非聚合直接跳转计算收益率

In [13]:

return2_data = ((market_data_indexed['close'].shift(-period) / market_data_indexed['close']).shift(0) - 1)
is_on_mark = return2_data.index.get_level_values(0).minute % int(period) == 0
return2_data = return2_data[is_on_mark]
return2_data.head()

trade_time
2022-07-25 09:45:00   -0.006684
2022-07-25 10:00:00   -0.000201
2022-07-25 10:15:00    0.002617
2022-07-25 10:30:00   -0.001434
2022-07-25 10:45:00    0.000431
Name: close, dtype: float64

In [43]:
market_data_indexed['ttime'] = market_data_indexed.index
return2_data = ((market_data_indexed['close'].shift(-period-1) / market_data_indexed['close']).shift(-1) - 1)

market_data_indexed['returns'] = return2_data

market_data_indexed.loc['2024-01-02 10:15:00']

code                        IM
close                   5877.6
high                    5877.6
low                     5871.0
open                    5872.6
volume                      70
ttime      2024-01-02 10:15:00
returns              -0.000783
Name: 2024-01-02 10:15:00, dtype: object

In [44]:
market_data_indexed['ttime'] = market_data_indexed.index
return2_data = ((market_data_indexed['close'].shift(-period) / market_data_indexed['close']).shift(0) - 1)

market_data_indexed['returns'] = return2_data

market_data_indexed.loc['2024-01-02 10:15:00']

code                        IM
close                   5877.6
high                    5877.6
low                     5871.0
open                    5872.6
volume                      70
ttime      2024-01-02 10:15:00
returns              -0.000136
Name: 2024-01-02 10:15:00, dtype: object

In [40]:
market_data_indexed['ttime'].shift(-period - 1)

trade_time
2022-07-25 09:31:00   2022-07-25 09:47:00
2022-07-25 09:32:00   2022-07-25 09:48:00
2022-07-25 09:33:00   2022-07-25 09:49:00
2022-07-25 09:34:00   2022-07-25 09:50:00
2022-07-25 09:35:00   2022-07-25 09:51:00
                              ...        
2024-05-29 13:21:00                   NaT
2024-05-29 13:22:00                   NaT
2024-05-29 13:23:00                   NaT
2024-05-29 13:24:00                   NaT
2024-05-29 13:25:00                   NaT
Name: ttime, Length: 107183, dtype: datetime64[ns]

In [41]:
market_data_indexed['ttime'].shift(- 1)

trade_time
2022-07-25 09:31:00   2022-07-25 09:32:00
2022-07-25 09:32:00   2022-07-25 09:33:00
2022-07-25 09:33:00   2022-07-25 09:34:00
2022-07-25 09:34:00   2022-07-25 09:35:00
2022-07-25 09:35:00   2022-07-25 09:36:00
                              ...        
2024-05-29 13:21:00   2024-05-29 13:22:00
2024-05-29 13:22:00   2024-05-29 13:23:00
2024-05-29 13:23:00   2024-05-29 13:24:00
2024-05-29 13:24:00   2024-05-29 13:25:00
2024-05-29 13:25:00                   NaT
Name: ttime, Length: 107183, dtype: datetime64[ns]

### 3.累计收益率

In [14]:
def create_chg(market_data, name='vwap'):
    pricep = market_data.set_index(['trade_time', 'code'])[name].unstack()
    pre_pricep = pricep.shift(1)
    ret_v2v = np.log((pricep) / pre_pricep)
    yields_data = ret_v2v.shift(-2)
    yields_data = yields_data.stack()
    yields_data.name = 'chg_pct'
    return yields_data.reset_index()

In [15]:
def create_yields(data, horizon, offset=0):
    df = data.copy()
    df.set_index("trade_time", inplace=True)
    ## chg为log收益
    df['nxt1_ret'] = df['chg_pct']
    df = df.groupby("code").rolling(
        window=horizon, min_periods=1)['nxt1_ret'].sum().groupby(level=0)
    df = df.shift(0).unstack().T.shift(-(horizon + offset - 1)).stack(
        dropna=False)
    df.name = 'nxt1_ret'
    return df

In [19]:
chg_data = create_chg(market_data = total_data, name='close')
df = create_yields(data=chg_data.copy(), horizon=period)
df.name = "nxt1_ret_{0}h".format(period)
returns3_data = df
is_on_mark = returns3_data.index.get_level_values(0).minute % int(period) == 0
return3_data = returns3_data[is_on_mark]
return3_data.head()
#chg_data.head()

trade_time           code
2022-07-25 09:45:00  IM     -0.005562
2022-07-25 10:00:00  IM      0.000029
2022-07-25 10:15:00  IM      0.000604
2022-07-25 10:30:00  IM     -0.000920
2022-07-25 10:45:00  IM      0.001466
Name: nxt1_ret_15h, dtype: float64

In [37]:
returns_data = fetch_temp_returns(
        method=method,
        instruments=instruments,
        datasets=['train'],
        category='returns')
returns4_data = returns_data.set_index('trade_time')[['nxt1_ret_{0}h'.format(period)]]
#is_on_mark = returns4_data.index.get_level_values(0).minute % int(period) == 0
#returns4_data = returns4_data[is_on_mark]
#returns4_data.head()

./records/cicso0/ims/returns/train_returns.feather


In [12]:
#returns_data#.loc['2024-01-02 10:15:00':].head(40)

In [13]:
returns4_data['nxt1_ret_{0}h'.format(period)].corr(
    return3_data
)

0.8250722921342027

In [14]:
returns4_data['nxt1_ret_{0}h'.format(period)].corr(
    return2_data
)

0.7113946049958093

In [15]:
returns3_data.corr(
    return2_data
)

0.7453942368816074